In [ ]:
import os

# Set both for redundancy
os.environ["HF_HOME"] = "/gpfs3/well/papiez/users/hri611/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/gpfs3/well/papiez/users/hri611/.cache/huggingfacee"
os.environ["HF_DATASETS_CACHE"] = "/gpfs3/well/papiez/users/hri611/.cache/huggingface/datasets"
os.environ["HF_METRICS_CACHE"] = "/gpfs3/well/papiez/users/hri611/.cache/huggingface/metrics"


In [ ]:
import io

import requests
import torch
import transformers
import glob
from pathlib import Path
import pydicom
import numpy as np
import io
import requests
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig, AutoTokenizer
import pandas as pd


In [ ]:
# step 1: Setup constant
device = "cuda"
dtype = torch.float16

# step 2: Load Processor and Model
processor = AutoProcessor.from_pretrained("StanfordAIMI/CheXagent-8b", trust_remote_code=True)
generation_config = GenerationConfig.from_pretrained("StanfordAIMI/CheXagent-8b")
model = AutoModelForCausalLM.from_pretrained("StanfordAIMI/CheXagent-8b", torch_dtype=dtype, trust_remote_code=True)


In [ ]:
root_dir = Path('/well/papiez/users/hri611/python/foundation-models-radiology')
dicom_paths = glob.glob(str(root_dir / 'PTX Head to Head Study Data' / '**/*.dcm'), recursive=True)

def dicom_to_rgb_image(dicom_path):
    dicom_image = pydicom.dcmread(dicom_path)
    pixel_array = dicom_image.pixel_array

    # Normalize to 0-255
    pixel_array = (pixel_array - np.min(pixel_array)) / (np.max(pixel_array) - np.min(pixel_array) + 1e-5)
    pixel_array = (pixel_array * 255.0).astype(np.uint8)

    # Convert grayscale to RGB
    if pixel_array.ndim == 2:
        image = Image.fromarray(pixel_array).convert("RGB")
    elif pixel_array.ndim == 3 and pixel_array.shape[-1] == 3:
        image = Image.fromarray(pixel_array)
    else:
        raise ValueError(f"Unexpected DICOM shape: {pixel_array.shape}")
    
    # save as jpg
    jpg_path = dicom_path.replace('.dcm', '.jpg')
    image.save(jpg_path, "JPEG")

# images = [dicom_to_rgb_image(dicom_paths[0])]
for dicom in dicom_paths:
    dicom_to_rgb_image(dicom)


In [ ]:
# 8B Model
# # step 3: Fetch the images
# # image_path = 'Pleural_effusion-Metastatic_breast_carcinoma_Case_166_(5477628658).jpg'
# # images = [Image.open(image_path).convert("RGB")]

# # step 4: Generate the Findings section
# dtype = model.dtype
# model.to(device)

# prompt = f'Does this chest X-ray contain a pneumothorax?'
# inputs = processor(images=images, text=f" USER: <s>{prompt} ASSISTANT: <s>", return_tensors="pt").to(device=device, dtype=dtype)
# output = model.generate(**inputs, generation_config=generation_config)[0]
# response = processor.tokenizer.decode(output, skip_special_tokens=True)
# print(response)

In [ ]:
# step 1: Setup constant
#bfloat16 (Brain Floating Point) is a 16-bit format optimized for deep learning:
#Faster and more memory-efficient than float32
model_name = "StanfordAIMI/CheXagent-2-3b"
dtype = torch.bfloat16
device = "cuda"

# step 2: Load Processor and Model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
model = model.to(device=device,dtype=dtype)
model.eval()

In [ ]:
responses = []
for i in range(len(jpg_paths)):  
    query = tokenizer.from_list_format([{'image': jpg_paths[i]},{'text': 'Does this chest X-ray contain a pneumothorax?'}])
    conv = [{"from": "system", "value": "You are a helpful assistant."}, {"from": "human", "value": query}]
    input_ids = tokenizer.apply_chat_template(conv, add_generation_prompt=True, return_tensors="pt")
    output = model.generate(
        input_ids.to(device), do_sample=False, num_beams=1, temperature=1., top_p=1., use_cache=True,
        max_new_tokens=512
    )[0]
    response = tokenizer.decode(output[input_ids.size(1):-1])
    responses.append(response)

In [ ]:
chexagent_df = pd.DataFrame({'image_path': jpg_paths, 'response': responses})

In [ ]:
chexagent_df['binary_score'] = chexagent_df['response'].apply(lambda x: 1 if 'yes' in x.lower() else 0)
chexagent_df.drop(columns=['response'], inplace=True)
chexagent_df.to_csv('ptx_chexagent_scores.csv', index=False)


In [ ]:
microsoft_df = pd.read_csv('ptx_biomedclip_scores.csv')
google_df = pd.read_csv('ptx_google_scores.csv')

merged_df = pd.merge(microsoft_df, google_df, on='image_paths', suffixes=('_microsoft', '_google'))

In [ ]:
merged_df

In [ ]:
merged_df['chexagent_scores'] = chexagent_df['binary_score']

In [ ]:
merged_df

In [ ]:
merged_df[['binary_score_google','binary_score_microsoft','chexagent_scores']].value_counts()